<a href="https://colab.research.google.com/github/softip/projeto_IA368X/blob/main/notebooks/Tratamentode_Dados_para_ML_random_florest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [90]:
import pandas as pd

#importa dataset do github
df = []
for i in range(1,10):
  df.append(pd.read_csv("https://raw.githubusercontent.com/softip/projeto_IA368X/main/data/external/dados_covid_ce_{}.csv".format(i)))

dfTotal = pd.DataFrame()
dfTotal = pd.concat(df)
dfTotal.reset_index()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (35,36,37,38,40) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (39,40) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (15,24,25,26,39,40) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (40) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result

,index,codigoPaciente,paisPaciente,estadoPaciente,codigoMunicipioPaciente,municipioPaciente,sexoPaciente,idadePaciente,dataNotificacao,dataColetaExame,dataResultadoExame,resultadoFinalExame,dataInicioSintomas,tipoTesteExame,racaCorPaciente,municipioNotificacaoEsus,profissionalSaude,cnesNotificacaoEsus,idSivep,classificacaoEstadoSivep,dataInternacaoSivep,dataEntradaUtisSvep,bairroPaciente,comorbidadeNeurologiaSivep,dataSolicitacaoExame,obitoConfirmado,dataObito,tipoObito,dataNotificacaoObito,comorbidadeCardiovascularSivep,comorbidadeObesidadeSivep,evolucaoCaso,dataEvolucaoCasoSivep,comorbidadeHematologiaSivep,comorbidadeSindromeDownSivep,comorbidadeAsmaSivep,comorbidadeDiabetesSivep,comorbidadeImunodeficienciaSivep,comorbidadeRenalSivep,comorbidadePuerperaSivep,comorbidadePneumopatiaSivep,dataSaidaUtisSvep
0,0,579fce2867018c510bdfbc9c64c14a34,BRASIL,CE,230490.0,GROAIRAS,FEMININO,25.0,2020-01-01T03:00:00.000+0000,2021-01-08T03:00:00.000+0000,2021-01-08T03:00:00.000+0000,Negativo,2020-01-01T03:00:00.000+0000,Teste Rápido - Anticorpo,Parda,GROAIRAS,0.0,6565239,3.159248e+11,Não realizado.0,2020-06-18T03:00:00.000+0000,2020-06-18T03:00:00.000+0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,504dc6479d137aef7556a9878626aef8,NaN,CE,230240.0,BOA VIAGEM,MASCULINO,39.0,2020-01-01T03:00:00.000+0000,2020-10-20T03:00:00.000+0000,2020-10-20T03:00:00.000+0000,Positivo,2020-01-01T03:00:00.000+0000,Teste Rápido - Anticorpo,Parda,MARACANAU,0.0,NaN,NaN,NaN,NaN,NaN,ALTO DO MOTOR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,9b1fca71df2ed7f6bcbc9f20122c78d2,BRASIL,CE,230075.0,AMONTADA,MASCULINO,66.0,2020-01-02T03:00:00.000+0000,2021-01-02T03:00:00.000+0000,2021-01-02T03:00:00.000+0000,Negativo,2020-01-02T03:00:00.000+0000,Teste Rápido - Anticorpo,Parda,AMONTADA,0.0,6460038,3.161099e+11,Negativo.0,2021-01-02T03:00:00.000+0000,NaN,NaN,Sim,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,fd948fcaca62c97a2f57a1717e7ad128,NaN,CE,230170.0,AURORA,MASCULINO,57.0,2020-01-02T03:00:00.000+0000,2020-01-02T03:00:00.000+0000,2020-01-02T03:00:00.000+0000,Positivo,2020-01-02T03:00:00.000+0000,Teste Rápido - Antígeno,Parda,PRINCESA ISABEL,0.0,NaN,NaN,NaN,NaN,NaN,AURORA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,fdccba56299c30f73a717ec52e5fc740,NaN,CE,231030.0,PARAMBU,FEMININO,25.0,2020-01-04T03:00:00.000+0000,2021-01-08T03:00:00.000+0000,2021-01-08T03:00:00.000+0000,Positivo,2020-01-01T03:00:00.000+0000,RT-PCR,Parda,PARAMBU,0.0,6362869,NaN,NaN,NaN,NaN,CAIXA D`AGUA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1873578,167232,eefa5ea801b53366cce43b2b09a58bdf,NaN,CE,230440.0,FORTALEZA,FEMININO,3.0,2021-05-21T03:00:00.000+0000,NaN,NaN,NaN,2021-05-15T03:00:00.000+0000,NaN,Parda,FORTALEZA,0.0,407836,NaN,NaN,NaN,NaN,PICI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1873579,167233,ab18e9b4902269f89d864e17da48e35f,NaN,CE,231290.0,SOBRAL,FEMININO,16.0,2021-05-19T03:00:00.000+0000,NaN,NaN,NaN,2021-05-17T03:00:00.000+0000,NaN,Parda,SOBRAL,1.0,2.42412e+06,NaN,NaN,NaN,NaN,PEDRINHAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1873580,167234,ad2979dd853f247793d4ec2fb967be3b,BRASIL,CE,230440.0,FORTALEZA,MASCULINO,43.0,2021-05-06T03:00:00.000+0000,2021-04-26T03:00:00.000+0000,2021-04-28T04:26:39.000+0000,Positivo,2021-05-04T03:00:00.000+0000,RT-PCR,Parda,FORTALEZA,0.0,7.99322e+06,3.162033e+11,Negativo.0,NaN,2021-05-06T03:00:00.000+0000,SIQUEIRA,NaN,2021-04-26T03:00:00.000+0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sim,NaN,NaN,NaN,NaN,NaN,NaN
1873581,167235,114849ef4d40d96f060316c8be22abf3,NaN,CE,230370.0,CAUCAIA,MASCULINO,33.0,2021-05-09T03:00:00.000+0000,NaN,NaN,NaN,2021-05-07T03:00:0

In [91]:
#mostra os campos e a quantidade de dados faltantes
dfTotal.isna().sum()

codigoPaciente                            0
paisPaciente                        1786289
estadoPaciente                          256
codigoMunicipioPaciente               14685
municipioPaciente                        40
sexoPaciente                             52
idadePaciente                            80
dataNotificacao                           0
dataColetaExame                      375130
dataResultadoExame                   379586
resultadoFinalExame                  117663
dataInicioSintomas                    17533
tipoTesteExame                       117983
racaCorPaciente                       95777
municipioNotificacaoEsus              30541
profissionalSaude                     30503
cnesNotificacaoEsus                  385965
idSivep                             1791914
classificacaoEstadoSivep            1805751
dataInternacaoSivep                 1801016
dataEntradaUtisSvep                 1855513
bairroPaciente                        19590
comorbidadeNeurologiaSivep      

In [92]:
#mostra a quantidade de registros no dataset
dfTotal.shape

(1873583, 41)

**Seleciona apenas as colunas de interesse**

In [93]:
#Define as colunas de interesse
features = ["codigoPaciente",
"codigoMunicipioPaciente",
"sexoPaciente",
"idadePaciente",
"racaCorPaciente",
"comorbidadeNeurologiaSivep",
"comorbidadeCardiovascularSivep",
"comorbidadeObesidadeSivep",
"comorbidadeHematologiaSivep",
"comorbidadeSindromeDownSivep",
"comorbidadeAsmaSivep",
"comorbidadeDiabetesSivep",
"comorbidadeImunodeficienciaSivep",
"comorbidadeRenalSivep",
"comorbidadePuerperaSivep",
"comorbidadePneumopatiaSivep",
"dataNotificacao",
"resultadoFinalExame",
"dataInternacaoSivep",
"dataEntradaUtisSvep",
"obitoConfirmado",
"dataObito",
"tipoObito"]

#seleciona apenas as colunas de interesse
dfTotalColunasEspecificas = dfTotal.loc[:,features]

In [94]:
dfTotalColunasEspecificas = dfTotal #corrigir isso aqui

**Segundo Filtro: Selecionar apenas os pacientes que vieram a óbito**

In [95]:
dfObitosGeral = dfTotalColunasEspecificas.loc[dfTotalColunasEspecificas.dataObito.notna()]

In [96]:
#mostra o número total de registros
dfObitosGeral.shape

(24914, 41)

**Remove do Dataset os óbitos que ainda estão em investigação**

In [97]:
#remove os óbitos em investigação
dfObitosConfirmados = dfObitosGeral.loc[~dfObitosGeral.tipoObito.eq("Em investigação")]


In [98]:
#mostra o número de registros após a filtragem
dfObitosConfirmados.shape

(23896, 41)

**Remove os pacientes duplicados**

In [99]:
#Remove as linhas duplicadas
dfObitosNonLinesDuplicates = dfObitosConfirmados.drop_duplicates(subset=["codigoPaciente"], keep="last")

In [100]:
#mostra o número de registros após a filtragem
dfObitosNonLinesDuplicates.shape

(20122, 41)

**Preenche as campos vázios das combormidades com Não**

In [101]:
#seleciona apenas as features que possuem comorbidades
comorbidade_cols = [col for col in dfObitosNonLinesDuplicates.columns if 'comorbidade' in col]

#Preenche as comorbidades com valores vázios como se não tivessem a comorbidades
dfObitosNonLinesDuplicates.loc[:,comorbidade_cols] = dfObitosNonLinesDuplicates.loc[:, comorbidade_cols].fillna("Não")

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1715: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, v)


**Preenche a coluna obitoConfirmado com valor  Não se o valor for vázio. A coluna obitoConfirmado indica se a pessoa morreu por Covid. Os valores nulos para essa coluna correspondem aos casos descartados (tipoObito).**

In [102]:
#Existe 1616 obitos que foram descartados e estão com valor nulo na coluna obitoConfirmado
dfObitosNonLinesDuplicates[dfObitosNonLinesDuplicates.tipoObito.eq("Descartado")].obitoConfirmado.value_counts(dropna=False)

NaN    1616
Name: obitoConfirmado, dtype: int64

In [103]:
#Preenche com 0 (Não) a coluna obitoConfirmado se ela estiver vázia
dfObitosNonLinesDuplicates.obitoConfirmado.fillna(0, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


**Remove o único paciente sem idade**

In [104]:
#remove
dfObitosNonLinesDuplicates.dropna(subset=["idadePaciente"], inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [105]:
#mostra o número de registros após a filtragem
dfObitosNonLinesDuplicates.shape

(20121, 41)

**Remove os pacientes sem código de Municipio - provavelmente não são do Ceará**

In [106]:
dfObitosNonLinesDuplicates.dropna(subset=["codigoMunicipioPaciente"], inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [107]:
#mostra o número de registros após a filtragem
dfObitosNonLinesDuplicates.shape

(20087, 41)

**Remove os paciente que não possuem a raça registrada**

In [108]:
#remove os pacientes que não possuem a raça informada
dfObitosNonLinesDuplicates.dropna(subset=["racaCorPaciente"], inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [109]:
#mostra o número de registros após a filtragem
dfObitosNonLinesDuplicates.shape

(13914, 41)

**Verifica a quantidade de registros faltantes**

In [110]:
dfObitosNonLinesDuplicates.isna().sum()

codigoPaciente                          0
paisPaciente                         1904
estadoPaciente                          0
codigoMunicipioPaciente                 0
municipioPaciente                       0
sexoPaciente                            0
idadePaciente                           0
dataNotificacao                         0
dataColetaExame                       426
dataResultadoExame                   2661
resultadoFinalExame                     0
dataInicioSintomas                      4
tipoTesteExame                         29
racaCorPaciente                         0
municipioNotificacaoEsus             4948
profissionalSaude                    4947
cnesNotificacaoEsus                  6602
idSivep                              1905
classificacaoEstadoSivep             2915
dataInternacaoSivep                  3715
dataEntradaUtisSvep                  9876
bairroPaciente                       1439
comorbidadeNeurologiaSivep              0
dataSolicitacaoExame              

In [111]:
dfObitosNonLinesDuplicates.head()

,codigoPaciente,paisPaciente,estadoPaciente,codigoMunicipioPaciente,municipioPaciente,sexoPaciente,idadePaciente,dataNotificacao,dataColetaExame,dataResultadoExame,resultadoFinalExame,dataInicioSintomas,tipoTesteExame,racaCorPaciente,municipioNotificacaoEsus,profissionalSaude,cnesNotificacaoEsus,idSivep,classificacaoEstadoSivep,dataInternacaoSivep,dataEntradaUtisSvep,bairroPaciente,comorbidadeNeurologiaSivep,dataSolicitacaoExame,obitoConfirmado,dataObito,tipoObito,dataNotificacaoObito,comorbidadeCardiovascularSivep,comorbidadeObesidadeSivep,evolucaoCaso,dataEvolucaoCasoSivep,comorbidadeHematologiaSivep,comorbidadeSindromeDownSivep,comorbidadeAsmaSivep,comorbidadeDiabetesSivep,comorbidadeImunodeficienciaSivep,comorbidadeRenalSivep,comorbidadePuerperaSivep,comorbidadePneumopatiaSivep,dataSaidaUtisSvep
60,ae9baa0839895cf22975686c4b1a9326,BRASIL,CE,230440.0,FORTALEZA,MASCULINO,68.0,2020-02-01T03:00:00.000+0000,2021-02-01T06:00:00.000+0000,2021-02-04T09:25:43.000+0000,Positivo,2020-01-29T03:00:00.000+0000,RT-PCR,Amarela,NaN,NaN,NaN,3.161223e+11,Em Análise.0,2021-02-01T03:00:00.000+0000,NaN,QUINTINO CUNHA,Não,2021-02-02T03:00:00.000+0000,1.0,2021-02-04T03:00:00.000+0000,Confirmado,2021-02-04T16:37:14.483+0000,Sim,Sim,NaN,NaN,Não,Não,Não,Não,Não,Não,Não,Não,NaN
2864,2f9cc4a6ca7f010ce492345c88a22c2f,NaN,CE,230410.0,CRATEUS,FEMININO,36.0,2020-03-21T03:00:00.000+0000,2020-03-21T09:00:00.000+0000,2020-03-21T03:00:00.000+0000,Negativo,2020-03-06T03:00:00.000+0000,RT-PCR,Parda,CRATEUS,0.0,NaN,NaN,NaN,NaN,NaN,CATINGUEIRO,Não,NaN,0.0,2020-03-21T03:00:00.000+0000,Descartado,2020-05-20T20:24:37.783+0000,Não,Não,NaN,NaN,Não,Não,Não,Não,Não,Não,Não,Não,NaN
2865,d099b265a2d082750fca9a1809da83e2,BRASIL,CE,230440.0,FORTALEZA,MASCULINO,72.0,2020-03-21T03:00:00.000+0000,2020-03-20T09:00:00.000+0000,2020-04-20T03:00:00.000+0000,Positivo,2020-03-15T03:00:00.000+0000,RT-PCR,Parda,FORTALEZA,0.0,NaN,3.158522e+11,Positivo.0,2020-03-20T03:00:00.000+0000,2020-03-21T03:00:00.000+0000,DAMAS,Não,NaN,1.0,2020-03-26T03:00:00.000+0000,Confirmado,2020-05-20T20:24:35.363+0000,Sim,Não,Óbito.0,2020-03-26T03:00:00.000+0000,Não,Não,Não,Sim,Não,Não,Não,Não,NaN
3315,9024e07873bb2af3780d023956489d44,BRASIL,CE,230080.0,ANTONINA DO NORTE,MASCULINO,85.0,2020-03-22T03:00:00.000+0000,2021-01-14T12:00:00.000+0000,2020-04-28T03:00:00.000+0000,Positivo,2020-03-18T03:00:00.000+0000,RT-PCR,Parda,JUAZEIRO DO NORTE,0.0,NaN,3.161048e+11,Positivo.0,NaN,NaN,NaN,Não,NaN,1.0,2021-01-14T03:00:00.000+0000,Confirmado,2021-01-19T14:22:02.803+0000,Sim,Não,Óbito.0,2021-01-12T03:00:00.000+0000,Não,Não,Não,Sim,Não,Não,Não,Sim,NaN
5137,cfcbf0b29e7a75ca9af3c37e11d5d8c0,BRASIL,CE,230590.0,IPUEIRAS,FEMININO,84.0,2020-03-24T03:00:00.000+0000,2020-03-24T15:00:00.000+0000,2020-04-28T03:00:00.000+0000,Positivo,2020-03-19T03:00:00.000+0000,RT-PCR,Parda,CRATEUS,0.0,NaN,3.158558e+11,Positivo.0,2020-03-23T03:00:00.000+0000,NaN,TUCUNS,Não,NaN,1.0,2020-03-25T03:00:00.000+0000,Confirmado,2020-05-19T14:59:55.730+0000,Sim,Não,Óbito.0,2020-03-26T03:00:00.000+0000,Não,Não,Não,Sim,Não,Não,Não,Não,NaN


**Altera valor das colunas comorbidades para númerico**

In [112]:
#seleciona apenas as features que possuem comorbidades
comorbidade_cols = [col for col in dfObitosNonLinesDuplicates.columns if 'comorbidade' in col]
#Substitui Ignorado e Não por 0 e Sim por 1
for c in comorbidade_cols:
  dfObitosNonLinesDuplicates.loc[dfObitosNonLinesDuplicates[c].eq("Ignorado") | dfObitosNonLinesDuplicates[c].eq("Não") | dfObitosNonLinesDuplicates[c].isna() ,c] = 0
  dfObitosNonLinesDuplicates.loc[dfObitosNonLinesDuplicates[c].eq("Sim"),c] = 1

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


**Altera o sexo para númerico**

In [113]:
dfObitosNonLinesDuplicates.loc[dfObitosNonLinesDuplicates.sexoPaciente.eq("FEMININO"),"sexoPaciente"] = 0
dfObitosNonLinesDuplicates.loc[dfObitosNonLinesDuplicates.sexoPaciente.eq("MASCULINO"),"sexoPaciente"] = 1

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


**Alteração da raça para númerico**

In [114]:
dfObitosNonLinesDuplicates.loc[dfObitosNonLinesDuplicates.racaCorPaciente.eq("Parda"),"racaCorPaciente"] = 0
dfObitosNonLinesDuplicates.loc[dfObitosNonLinesDuplicates.racaCorPaciente.eq("Branca"),"racaCorPaciente"] = 1
dfObitosNonLinesDuplicates.loc[dfObitosNonLinesDuplicates.racaCorPaciente.eq("Preta"),"racaCorPaciente"] = 2
dfObitosNonLinesDuplicates.loc[dfObitosNonLinesDuplicates.racaCorPaciente.eq("Amarela"),"racaCorPaciente"] = 3
dfObitosNonLinesDuplicates.loc[dfObitosNonLinesDuplicates.racaCorPaciente.eq("Indígena"),"racaCorPaciente"] = 4

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [115]:
dfObitosNonLinesDuplicates.loc[dfObitosNonLinesDuplicates.racaCorPaciente.eq("Ignorado"),"racaCorPaciente"] = 5

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


**Verifica os valores únicos para todas colunas**

In [116]:
colunas = comorbidade_cols + ["sexoPaciente", "idadePaciente", "racaCorPaciente", "obitoConfirmado", "tipoObito"]
for c in colunas:
  print(c, ":", dfObitosNonLinesDuplicates[c].unique())

comorbidadeNeurologiaSivep : [0 1]
comorbidadeCardiovascularSivep : [1 0]
comorbidadeObesidadeSivep : [1 0]
comorbidadeHematologiaSivep : [0 1]
comorbidadeSindromeDownSivep : [0 1]
comorbidadeAsmaSivep : [0 1]
comorbidadeDiabetesSivep : [0 1]
comorbidadeImunodeficienciaSivep : [0 1]
comorbidadeRenalSivep : [0 1]
comorbidadePuerperaSivep : [0 1]
comorbidadePneumopatiaSivep : [0 1]
sexoPaciente : [1 0]
idadePaciente : [ 68.  36.  72.  85.  84.  64.  80.  86.  77.  98.  41.   1.  76.  90.
  69.  39.  42.  83.  78.  67.  58.  75.  66.  93.  51.  74.  65.  60.
  55.  81.  79.  82.  40.  71.  45.  59.  87.  94.  53.  46.  92.  70.
  73.  33.  57.  56.  62.  54.  89.  88.  49.  44.  23.  91.  96.  35.
  61.  95.  47.  63.  38.  17.  99. 101.  52. 100.  10.  97.  50.  43.
  30.  37.  26.  21.  19.  32.   3.  48.  29.  16.  34. 104.   0.  18.
  15.   7.  28.   2.  25. 103.   9. 102.  13.  24.  22.  31.  20. 110.
 107.  27.  11.   5.   6. 105.  14.   4. 116.   8. 114. 106.]
racaCorPaciente : [3 

**Agrupa a idade em categorias de 5 anos**

In [117]:
bins = [x for x in range(0,110, 10)]
labels = ["{} a {}".format(x, x+9) for x in range(0,100, 10)]
labels[-1] = 'mais de 90'
dfObitosNonLinesDuplicates['idadeCategoria'] = pd.cut(dfObitosNonLinesDuplicates['idadePaciente'], bins=bins, labels=labels)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [118]:
#Prenche com a categoria mais de 90 anos quando as idades foram maiores que 100 (bins pega até de 90 a 100)
dfObitosNonLinesDuplicates.idadeCategoria.fillna("mais de 90", inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


**Mostra a distribuição dos dados**

In [119]:
dfObitosNonLinesDuplicates.obitoConfirmado.value_counts()

1.0    12618
0.0     1296
Name: obitoConfirmado, dtype: int64

**Corrige o nome das colunas comorbidades**

In [120]:
#Melhora a visualização dos nomes das comorbidades
def ajustaNome(x):
  x = x.replace("comorbidade", "")
  return x.replace("Sivep", "")

cols_dic = {c:ajustaNome(c) for c in comorbidade_cols}

In [121]:
dfObitosNonLinesDuplicates.rename(columns = cols_dic, inplace = True) 

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


**Abre dados do IBGE relativo aos município dos paciente**

In [122]:
import numpy as np
#abre dataset IBGE
dfIbge = pd.read_csv("https://raw.githubusercontent.com/softip/projeto_IA368X/main/data/external/dados_ibge.csv")
#remove o último digito da coluna código do Ibge, para ficar igual ao dataset da covid
dfIbge['codigo_ibge_6d'] = dfIbge['codigo_ibge'].astype(str).str[:-1].astype(np.int64)

In [123]:
#corrige o tipo do campo codigoMunicipio de float para int
dfObitosNonLinesDuplicates["codigoMunicipioPaciente"] = dfObitosNonLinesDuplicates["codigoMunicipioPaciente"].astype(int)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [124]:
#junta os dataset
dfPacienteIBGE = pd.merge(dfObitosNonLinesDuplicates, dfIbge, how="inner", left_on="codigoMunicipioPaciente", right_on="codigo_ibge_6d")

**Filtra apenas as colunas que serão utilizadas no modelo**

In [125]:
use_feature = ["sexoPaciente","idadePaciente","racaCorPaciente","Asma","Cardiovascular","Diabetes","Hematologia","Imunodeficiencia","Neurologia","Obesidade","Pneumopatia","Puerpera","Renal","SindromeDown","area_territorial_km2", "populacao_estimada_2020", "densidade_demografica_hab_km2_2010", "escolarizacao6a14 anos_2010", "IDHM_2010", "mortalidade_infantil_2019", "PIB_per_capita_2018","obitoConfirmado"]
dfFinal = dfPacienteIBGE.loc[:,use_feature]

**Confirma se não existes valores nulos**

In [126]:
dfFinal.isna().sum()

sexoPaciente                          0
idadePaciente                         0
racaCorPaciente                       0
Asma                                  0
Cardiovascular                        0
Diabetes                              0
Hematologia                           0
Imunodeficiencia                      0
Neurologia                            0
Obesidade                             0
Pneumopatia                           0
Puerpera                              0
Renal                                 0
SindromeDown                          0
area_territorial_km2                  0
populacao_estimada_2020               0
densidade_demografica_hab_km2_2010    0
escolarizacao6a14 anos_2010           0
IDHM_2010                             0
mortalidade_infantil_2019             0
PIB_per_capita_2018                   0
obitoConfirmado                       0
dtype: int64

**Salva o dataset**

In [128]:
dfFinal.to_csv("DataSetCovidIBGE_RandomFlorest.csv", index=False)